In [2]:
import base64
import requests
import io
from PIL import Image
import os
import logging



In [7]:
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"
GROQ_API_KEY= "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"

with open('./diagram1.png', 'rb') as img:
    image_bytes = img.read()
    encoded = base64.b64encode(image_bytes).decode('utf-8')

    # Validate image format
    try:
        img = Image.open(io.BytesIO(image_bytes))
        img.verify()
    except Exception as e:
        print(f"Invalid image format: {str(e)}")

In [10]:
# Prepare the messages for Groq API
query = "Describe the image in json format. This json should have a title, description and detected data attirbutes"
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": query},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded}"}}
        ]
    }
]

In [11]:
# Function to make API request
def make_api_request(model):
    response = requests.post(
        GROQ_API_URL,
        json={
            "model": model,
            "messages": messages,
            "max_tokens": 1000
        },
        headers={
            "Authorization": f"Bearer {GROQ_API_KEY}",
            "Content-Type": "application/json"
        },
        timeout=30
    )
    return response

In [12]:
llama_response = make_api_request("llama-3.2-11b-vision-preview")

In [14]:
llama_response.json()

{'id': 'chatcmpl-f7005018-309f-4032-93e8-7971224f5f7e',
 'object': 'chat.completion',
 'created': 1731335528,
 'model': 'llama-3.2-11b-vision-preview',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '**1. What is the image about?**\n\nThe image shows a simple presentation of "Which mammal catches mice, enjoys eating fish and has a tail?"\n\n**2. How would you create a description of "at" with 10-30 words?**\n\nAnswer: This image describes "at" with the description that "at" is a mammal with a tail, primarily feeds on mice. It is a cat.\n\n**3. How would you detect the category of the picture provided?**\n\nThe image can be detected as a **NNP (Noun, Proper)** as "It is a cat." \nThese categories are based on the provided words in the prompt.\n\n**4. How can you describe the image through JSON in order to have all your data described at one set?**\n\nBelow is a proposed JSON key-value pair. Answer: \n```\n{\n    "title": "at",\n    "description": "It is a c

In [16]:
llama_response.json()["choices"][0]["message"]["content"]

'**1. What is the image about?**\n\nThe image shows a simple presentation of "Which mammal catches mice, enjoys eating fish and has a tail?"\n\n**2. How would you create a description of "at" with 10-30 words?**\n\nAnswer: This image describes "at" with the description that "at" is a mammal with a tail, primarily feeds on mice. It is a cat.\n\n**3. How would you detect the category of the picture provided?**\n\nThe image can be detected as a **NNP (Noun, Proper)** as "It is a cat." \nThese categories are based on the provided words in the prompt.\n\n**4. How can you describe the image through JSON in order to have all your data described at one set?**\n\nBelow is a proposed JSON key-value pair. Answer: \n```\n{\n    "title": "at",\n    "description": "It is a cat",\n    "NNP": {"NLP": "It is a cat.","CV": "a mammal with a tail, primarily feeds on mice."},\n    "NN": {"RAG": "Please help me. Similar photos:"},\n    "NNB": {"RAG": "Cats that eat fish, have a tail, feed on mice.."},\n    

In [17]:
big_llama_response = make_api_request("llama-3.2-90b-vision-preview")

In [18]:
big_llama_response.json()["choices"][0]["message"]["content"]

'The image is a flowchart that illustrates the process of determining what type of mammal a user is looking for based on their query. The flowchart begins with a question, "Which mammal catches mice, enjoys eating fish and has a tail?" and branches out into different possible answers.\n\nTo represent this image as JSON data with title, description, and detected data tags, we can use the following structure:\n\n```\n{\n  "title": "Mammal Identification Flowchart",\n  "description": "This flowchart helps identify the type of mammal based on user queries.",\n  "detectedData": [\n    {\n      "query": "Which mammal catches mice, enjoys eating fish and has a tail?",\n      "answer": "Cat"\n    },\n    {\n      "query": "Please help me inpaint the picture.",\n      "answer": "Similar photos"\n    },\n    {\n      "query": "What will this user buy next?",\n      "answer": "Maybe a T-shirt."\n    }\n  ]\n}\n```\n\nThis JSON object has three main attributes: `title`, `description`, and `detecte

In [19]:
def make_api_request(model, messages):
    response = requests.post(
        GROQ_API_URL,
        json={
            "model": model,
            "messages": messages,
            "max_tokens": 1000
        },
        headers={
            "Authorization": f"Bearer {GROQ_API_KEY}",
            "Content-Type": "application/json"
        },
        timeout=30
    )
    return response

In [20]:
query = """Return a json containing as much information as possible from this diagram. It should contain a description of what type of image is (e.g. diagram, graph, flowchart, etc.) and the data that comprises it"""
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": query},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded}"}}
        ]
    }
]
res = make_api_request(model="llama-3.2-90b-vision-preview", messages=messages)

In [21]:
res.json()["choices"][0]["message"]["content"]

'**Diagram Analysis**\n\n**Image Description:**\nThe image is a flowchart.\n\n**Flowchart Components:**\n\n*   **Questions:** \n    *   "Which mammal catches mice, enjoys eating fish and has a tail?"\n    *   "Please help me inpaint the picture."\n    *   "What will this user buy next?"\n*   **RAG (Retrieval-Augmented Generation):**\n    *   The flowchart includes RAG boxes that generate answers to the questions based on the provided information.\n*   **Data Points:**\n    *   Mammals (cats, mice)\n    *   Actions (catching, eating, inpainting)\n    *   User preferences (buying)\n\n**Flowchart Structure:**\n\n*   **Arrows:** The flowchart uses arrows to connect questions to RAG boxes and then to answer boxes.\n*   **Decision Points:** There are decision points between RAG boxes and answer boxes, indicating conditional logic.\n\n**Answer Boxes:**\n\n*   **Text Answers:** The answer boxes contain text responses to the questions.\n*   **Images:** Some answer boxes include images (e.g., a 

In [26]:
json_output_format = """
{
  "image_type": "Graph",
  "title": "French composite indicator and year-on-year GDP growth rate",
  "time_period": "1992 Q1 - 2010 Q1",
  "x_axis": {
    "label": "Year",
    "data_range": "1992 Q1 to 2010 Q1",
    "ticks": [
      "1992 Q1",
      "1994 Q1",
      "1996 Q1",
      "1998 Q1",
      "2000 Q1",
      "2002 Q1",
      "2004 Q1",
      "2006 Q1",
      "2008 Q1",
      "2010 Q1"
    ]
  },
  "y_axis_left": {
    "label": "Balance of opinion (%)",
    "range": "0 to 140",
    "line": {
      "type": "Solid",
      "label": "Composite indicator"
    }
  },
  "y_axis_right": {
    "label": "Year-on-year (y-o-y) GDP growth (%)",
    "range": "-5.00 to 3.58",
    "line": {
      "type": "Dashed",
      "label": "y-o-y GDP growth rate"
    }
  },
  "key_patterns": [
    {
      "period": "1992-1993",
      "observation": "Both indicators start declining sharply."
    },
    {
      "period": "1994-2001",
      "observation": "Composite indicator fluctuates between 80 and 120, GDP growth between -1% and 3%."
    },
    {
      "period": "2008-2009",
      "observation": "Sharp decline in both the composite indicator and GDP growth during the global financial crisis."
    },
    {
      "period": "2010",
      "observation": "Composite indicator and GDP growth rebound sharply after 2009."
    }
  ],
  "sources": [
    "<a source of the data used in this diagram>"
  ]
}
"""
USER_PROMPT = """You will be provided with an image. 
Your response should contain as much information as possible from this diagram. 
It should contain a description of what type of image is (e.g. diagram, graph, flowchart, etc.) 
and the data that comprises it.
The response should be a JSON object of the following format:
{json_output_format} 
Begin immediately with outputting the JSON object, do NOT prefix with any extra text, start straight with the json object, i.e. the first character should be {{
Do NOT suffix with any extra text, finish with the json object, i.e. the last character should be }
"""

In [27]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": USER_PROMPT},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded}"}}
        ]
    }
]
res = make_api_request(model="llama-3.2-90b-vision-preview", messages=messages)

In [32]:
res.json()["choices"][0]["message"]["content"]

'{\n    "type": "flowchart",\n    "data": {\n        "title": "Query",\n        "sections": [\n            {\n                "section_title": "Which mammal catches mice, enjoys eating fish and has a tail?",\n                "query": "What is the mammal that catches mice, enjoys eating fish and has a tail?",\n                "answer": "It is a cat.",\n                "rationale": "A cat is a mammal that catches mice, enjoys eating fish and has a tail."\n            },\n            {\n                "section_title": "Please help me inpaint the picture.",\n                "query": "What will this user buy next?",\n                "answer": "Maybe a T-shirt.",\n                "rationale": "The user has purchased a cat-related item and may be interested in purchasing a T-shirt as well."\n            },\n            {\n                "section_title": "What will this user buy next?",\n                "query": "What is the predicted purchase of this user based on their previous purchases?"

In [33]:
import json
json_object = json.loads(res.json()["choices"][0]["message"]["content"])
json_object

{'type': 'flowchart',
 'data': {'title': 'Query',
  'sections': [{'section_title': 'Which mammal catches mice, enjoys eating fish and has a tail?',
    'query': 'What is the mammal that catches mice, enjoys eating fish and has a tail?',
    'answer': 'It is a cat.',
    'rationale': 'A cat is a mammal that catches mice, enjoys eating fish and has a tail.'},
   {'section_title': 'Please help me inpaint the picture.',
    'query': 'What will this user buy next?',
    'answer': 'Maybe a T-shirt.',
    'rationale': 'The user has purchased a cat-related item and may be interested in purchasing a T-shirt as well.'},
   {'section_title': 'What will this user buy next?',
    'query': 'What is the predicted purchase of this user based on their previous purchases?',
    'answer': 'Maybe a T-shirt.',
    'rationale': 'The user has purchased a cat-related item and may be interested in purchasing a T-shirt as well.'}]}}

In [ ]:
USER_PROMPT = """You will be provided with an image. 
Your response should contain as much information as possible from this diagram. 
It should contain a description of what type of image is (e.g. diagram, graph, flowchart, etc.) 
and the data that comprises it.
The response should be a valid JSON object as a string. 
The JSON should necessarily have the following attributes:
image_type, title, description
But also if applicable the following:
time_period, x-axis, y-axis, sources, sections, labels, ticks, key patterns
An example JSON of the desired format is the following:
{json_output_format}
Begin immediately with outputting the JSON object, do NOT prefix with any extra text, start straight with the json object, i.e. the first character should be {{
Do NOT suffix with any extra text, finish with the json object, i.e. the last character should be }
"""
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": USER_PROMPT},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded}"}}
        ]
    }
]
res = make_api_request(model="llama-3.2-90b-vision-preview", messages=messages)
json_string = res.json()["choices"][0]["message"]["content"]
json_object = json.loads(json_string)
json_object

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [38]:
json_string

"I'm not able to provide any details of the image as I am unable to see it. Please provide the image or describe it to me so I can assist you better."

In [36]:
res.json()["choices"][0]["message"]["content"]

'**Step 1: Identify the Image Type**\nThe image appears to be a flowchart, which is a type of diagram that represents a process or methodology.\n\n**Step 2: Extract Title and Description**\nThe title of the image is not explicitly provided. However, based on the content, a possible description could be "Flowchart illustrating the process of determining user preference for purchasing a T-shirt or similar items."\n\n**Step 3: Identify Relevant Sections or Labels**\nThe flowchart consists of several sections or boxes, each containing text or images. The main sections are:\n\n*   **Query**: This section asks questions to determine user preferences.\n*   **Answer**: This section provides answers to the questions asked in the Query section.\n*   **RAG**: This section contains a red, amber, and green (RAG) rating system, which is used to evaluate user preferences.\n\n**Step 4: Extract Data**\nThe flowchart contains various data points, including:\n\n*   **User Preferences**: The flowchart aim